In [3]:
import os
#os.chdir("firecitadel")

from utils import *
import json, logging, random

logger = logging.getLogger()
logger.setLevel(logging.INFO)


In [4]:

# Configure the root logger to log at the "INFO" level
logging.basicConfig(level=logging.INFO)

with open("templates.json", "r+") as fp:
    templates = json.load(fp)
    styles = templates["styles"]
    environments = templates["environments"]

style = random.choice(styles)
environment = random.choice(environments)

In [26]:
style = "Banana"
environment = "Casino"
logging.info(f"Generating image with {style} style, in a {environment} environment")

INFO:root:Generating image with Banana style, in a Casino environment


In [25]:
story_chain = create_storyteller_chain(temperature = 1.2)

def more_than_n_characters(text, n = 280):
    return len(text) > n

# Generate Story
cant_be_tweeted = True
while cant_be_tweeted:
    story = story_chain.run({"style" : style, "environment" : environment})
    if cant_be_tweeted := more_than_n_characters(story):
        logging.warning(f"The story has {len(story)} characters. Trying again")
    else:
        logging.info(f"Story: {story}")
    

INFO:root:Story: The banana was feeling lucky, so it put its peel out on the floor. As the gambler approached, they slipped and knocked over the entire slot machine row, causing chaos and a banana-filled triumph.


In [ ]:

# Generate Description of the story

img_chain = create_imagedescription_chain()
img_description = img_chain.run(story)
logging.info(f"Scene description: {img_description}")


In [18]:

# Generate image
image_urls = text_to_image(story, model = MODELS.DREAM_SHAPER)
url = image_urls[0]

logging.info(f"Generated image: {url}")

INFO:root:Generated image: https://pub-3626123a908346a7a8be8d9295f44e26.r2.dev/generations/0-02b3d597-513b-4ffe-9d3d-851debc7b701.png


In [19]:
from IPython.display import Image 
img = Image(url = url, width = 500)
display(img)

In [ ]:
# Upload image and publish tweet

tw_client = get_tweepy_client()
api = get_tweepy_API()

media = upload_image_to_twitter(url, api)

tweet_response = tw_client.create_tweet(
    text = story,
    media_ids = [media.media_id]
)

tweet_url = f"https://twitter.com/firecitadel/status/{tweet_response.data['id']}"
print(tweet_url)